In [1]:
%matplotlib inline
import tensorflow as tf
from os import system
import random
import vizdoom as vd
from vizdoom import *
import numpy as np
import scipy.ndimage as Simg
# import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from ennemies import *
import sys

from basic_ennemy_pos import basic_ennemy_x
from network import tf, DRQN
from video import VideoWriter
from memory import ReplayMemory
import map_parser
from config import (
    N_ACTIONS, LEARNING_RATE, MIN_MEM_SIZE, MAX_MEM_SIZE,
    MAX_CPUS, TRAINING_STEPS, BATCH_SIZE, SEQUENCE_LENGTH,
    QLEARNING_STEPS, MAX_EPISODE_LENGTH, DEATH_PENALTY,
    KILL_REWARD, PICKUP_REWARD, GREEDY_STEPS, IGNORE_UP_TO,
    BACKPROP_STEPS, USE_GAME_FEATURES, LEARN_Q, USE_RECURRENCE,
)
# from agent import *

In [3]:
visible = False
game = vd.DoomGame()
# game.load_config("basic.cfg")
game.load_config("scenarios/defend_the_line.cfg")
# game.load_config("scenarios/deadly_corridor.cfg")
game.set_screen_format(ScreenFormat.RGB24)
game.set_depth_buffer_enabled(True)

# Ennemy detection
walls = None  # map_parser.parse("maps/deathmatch.txt")
game.clear_available_game_variables()
game.add_available_game_variable(vd.GameVariable.POSITION_X)  # 0
game.add_available_game_variable(vd.GameVariable.POSITION_Y)  # 1
game.add_available_game_variable(vd.GameVariable.POSITION_Z)  # 2

game.add_available_game_variable(vd.GameVariable.KILLCOUNT)   # 3
game.add_available_game_variable(vd.GameVariable.DEATHCOUNT)  # 4
game.add_available_game_variable(vd.GameVariable.ITEMCOUNT)   # 5
game.add_available_game_variable(vd.GameVariable.ANGLE) # 6

game.set_labels_buffer_enabled(True)
game.set_window_visible(visible)

game.init()

In [ ]:
vd.GameVariable.ANGLE

In [6]:
state.game_variables[6]

0.0

In [9]:
walls = map_parser.parse("maps/defend_the_line.txt")

In [5]:
state=game.get_state()

In [11]:
has_visible_entities(state,walls)

[True, False, False]

In [7]:
for l in state.labels:
    print("Object id:", l.object_id, "object name:", l.object_name, "label:", l.value)
    print("Object position X:", l.object_position_x, "Y:", l.object_position_y, "Z:", l.object_position_z)

Object id: 1 object name: Demon label: 72
Object position X: 74.0 Y: 450.0 Z: 0.0
Object id: 2 object name: Demon label: 218
Object position X: 56.928924560546875 Y: 292.9289245605469 Z: 0.0
Object id: 3 object name: Demon label: 36
Object position X: 74.0 Y: 150.0 Z: 0.0
Object id: 4 object name: DoomImp label: 182
Object position X: 64.0 Y: 400.0 Z: 0.0
Object id: 5 object name: DoomImp label: 145
Object position X: 64.0 Y: 200.0 Z: 0.0
Object id: 6 object name: DoomImp label: 109
Object position X: 64.0 Y: 90.0 Z: 0.0
Object id: 0 object name: DoomPlayer label: 255
Object position X: -480.0 Y: 288.0 Z: 0.0


In [8]:
state.game_variables

array([-480.,  288.,    0.,    0.,    0.,    0.,    0.])

In [10]:
vd.GameVariable

vizdoom.vizdoom.GameVariable

In [31]:
relative_x = 0
relative_y = 0
min_dist = float('inf')
for obj in state.labels:
#     obj_pos_x = obj.object_position_x
#     obj_pos_y = obj.object_position_y
    a = np.array((obj.object_position_x ,obj.object_position_y))
    b = np.array((state.game_variables[0], state.game_variables[1]))
#     print(type(obj.object_name))
    if(obj.object_name != 'DoomPlayer'):
        dist = np.linalg.norm(a-b)
        if(dist < min_dist):
            min_dist = dist
            relative_x = obj.object_position_x - state.game_variables[0]
            relative_y = obj.object_position_y - state.game_variables[1]

In [26]:
relative_x

536.92892456054688

In [27]:
relative_y

4.928924560546875

In [28]:
min_dist

536.95154746687251

In [31]:
def min_relative_pos(state, entity_type):
    relative_x = 0
    relative_y = 0
    min_dist = float('inf')
    for obj in state.labels:
    #     obj_pos_x = obj.object_position_x
    #     obj_pos_y = obj.object_position_y
        if(obj.object_name not in entity_type):
            continue
        a = np.array((obj.object_position_x ,obj.object_position_y))
        b = np.array((state.game_variables[0], state.game_variables[1]))
    #     print(type(obj.object_name))
        if(obj.object_name != 'DoomPlayer'):
            dist = np.linalg.norm(a-b)
            if(dist < min_dist):
                min_dist = dist
                relative_x = obj.object_position_x - state.game_variables[0]
                relative_y = obj.object_position_y - state.game_variables[1]
                
#     return [relative_x, relative_y]
    if(relative_x == 0 and relative_y == 0):
        return 0
    else:
        return np.tanh(np.divide(relative_x, relative_y))


def get_min_relative_pos(state):
    types = ENNEMIES, PICKUPS, BLASTS
    return [min_relative_pos(state, x) for x in types]

def get_game_feature(state, wall):
    types = ENNEMIES, PICKUPS, BLASTS
    result = []
    for x in types:
        result.append(has_visible(state, wall, x))
#         result.append(min_relative_pos(state, x)[0])
#         result.append(min_relative_pos(state, x)[1])
        result.append(min_relative_pos(state, x))
    return result

In [48]:
from ennemies import *

In [32]:
get_game_feature(state, walls)

[True, 1.0, False, 0, False, 0]

In [37]:
entity_type = ENNEMIES
relative_x = 0
relative_y = 0
min_dist = float('inf')
for obj in state.labels:
#     obj_pos_x = obj.object_position_x
#     obj_pos_y = obj.object_position_y
    if(obj.object_name not in entity_type):
        continue
    a = np.array((obj.object_position_x ,obj.object_position_y))
    b = np.array((state.game_variables[0], state.game_variables[1]))
#     print(type(obj.object_name))
    if(obj.object_name != 'DoomPlayer'):
        dist = np.linalg.norm(a-b)
        if(dist < min_dist):
            min_dist = dist
            relative_x = obj.object_position_x - state.game_variables[0]
            relative_y = obj.object_position_y - state.game_variables[1]

#     return [relative_x, relative_y]
# if(relative_x == 0 and relative_y == 0):
#     return 0
# else:
#     return np.tanh(np.divide(relative_x, relative_y))

In [44]:
np.tanh(10)

0.99999999587769273

In [40]:
np.divide(relative_x, relative_y)

108.93429549690114

In [38]:
relative_x

536.92892456054688

In [39]:
relative_y

4.928924560546875

In [34]:
min_relative_pos(state, ENNEMIES)

1.0

In [59]:
True == 1

True

In [2]:
import math

In [4]:
math.tanh(536.92892456054688/4.928924560546875)

1.0

In [6]:
math.tanh(0)

0.0

In [15]:
tf.tanh(0.0)

<tf.Tensor 'Tanh_1:0' shape=() dtype=float32>

In [18]:
math.tanh(1)

0.7615941559557649

In [19]:
float('Inf')

inf

In [20]:
math.tanh(float('Inf'))

1.0

In [22]:
np.float64(1.0)/0.0

C:\_Software\Pro\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in double_scalars
  """Entry point for launching an IPython kernel.


inf

In [28]:
np.divide(-1,0)

C:\_Software\Pro\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


-inf

In [26]:
math.tanh(0)

0.0

In [30]:
np.tanh(np.divide(-1,0))

C:\_Software\Pro\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


-1.0